# VRPTW - M3AS

In [37]:
class Cliente:
    def __init__(self, numero, x, y, demanda, inicio, fin, tiempo):
        self.numero = int(float(numero))
        self.x = float(x)
        self.y = float(y)
        self.demanda = float(demanda)
        self.inicio = float(inicio)
        self.fin = float(fin)
        self.tiempo = float(tiempo)

In [42]:
# Preprocesar los datos de entrada

def procesar_instancia(filename):
    with open(filename, "r") as f:
        lineas = f.readlines()

    # Obtener clientes y capacidad
    clientes = int(lineas[1])
    capacidad = int(lineas[3])

    # Obtener por cada cliente los datos
    lista = []
    for cliente in lineas[5:]:
        lista.append(Cliente(*cliente.strip().split()))

    return capacidad, lista


In [44]:
capacidad_c101, clientes_c101 = procesar_instancia("datos/original/vrptw_c101.txt")
capacidad_rc101, clientes_rc101 = procesar_instancia("datos/original/vrptw_rc101.txt")